In [31]:
import pandas as pd
import random
from time import sleep
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
import numpy as np
import nltk
import os
from sklearn import datasets
import pickle
from nltk.tokenize import word_tokenize
from sklearn.decomposition import NMF, TruncatedSVD
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.neighbors import NearestNeighbors
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.linear_model import RidgeCV
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score, roc_curve, auc, fbeta_score
from sklearn.model_selection import learning_curve, KFold, GridSearchCV, cross_val_score
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.dummy import DummyClassifier
from collections import Counter
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction import text
from sklearn.preprocessing import normalize
import re
from unidecode import unidecode

import matplotlib.style as style
%matplotlib inline
plt.style.use('ggplot')

In [32]:
with open('data/cleaned_df.pkl', 'rb') as picklefile:
    df = pickle.load(picklefile)

In [33]:
df.head()

,song_owner,song_title,song_element,verse,artist
0,Childish Gambino,This Is America,[Bridge: Childish Gambino & Young Thug],"We just wanna party, Party just for you, We ju...",Childish Gambino
1,Childish Gambino,This Is America,[Chorus: Childish Gambino],"This is America, Don't catch you slippin' now,...",Childish Gambino
2,Childish Gambino,This Is America,"[Verse 1: Childish Gambino, Blocboy JB, Slim J...","This is America (skrrt, skrrt, woo), Don't cat...",Childish Gambino
3,Childish Gambino,This Is America,[Refrain: Choir & Childish Gambino],"Ooh-ooh-ooh-ooh-ooh, tell somebody, You go tel...",Childish Gambino
4,Childish Gambino,This Is America,"[Chorus: Childish Gambino, Slim Jxmmi & Young ...","This is America (woo, ayy), Don't catch you sl...",Childish Gambino


In [34]:
model_df = df.copy()

model_df = model_df[model_df['artist'] == 'Drake']

In [35]:
model_df['verse_words'] = model_df['verse'].copy()
model_df['verse_words'] = model_df['verse_words'].str.lower()
model_df['verse_words'] = model_df['verse_words'].apply(lambda x: x.replace(',',''))
model_df['verse_words'] = model_df['verse_words'].apply(lambda x: x.split(' '))

In [36]:
model_df.head()

,song_owner,song_title,song_element,verse,artist,verse_words
1282,Drake,Buried Alive (Interlude),[Intro],"If you was in a pine box, I would surely break...",Drake,"[if, you, was, in, a, pine, box, i, would, sur..."
1284,Drake,God's Plan,[Intro],"Yeah, they wishin' and wishin' and wishin' and...",Drake,"[yeah, they, wishin', and, wishin', and, wishi..."
1285,Drake,God's Plan,[Verse 1],"I been movin' calm, don't start no trouble wit...",Drake,"[i, been, movin', calm, don't, start, no, trou..."
1286,Drake,God's Plan,[Chorus 1],"God's plan, God's plan, I hold back, sometimes...",Drake,"[god's, plan, god's, plan, i, hold, back, some..."
1287,Drake,God's Plan,[Post-Chorus],"And still, bad things, It's a lot of bad thing...",Drake,"[and, still, bad, things, it's, a, lot, of, ba..."


In [37]:
from nltk.tokenize import word_tokenize
model_df['tokenize_verse'] = model_df['verse'].copy()
model_df['tokenize_verse'] = model_df['tokenize_verse'].str.lower()
model_df['tokenize_verse'] = model_df['tokenize_verse'].apply(lambda x: word_tokenize(x))

In [38]:
model_df['verse'] = model_df['verse'].str.lower()
model_df['verse'] = model_df['verse'].apply(lambda x: x.replace("'",''))
model_df['verse'] = model_df['verse'].apply(lambda x: "".join([c if c.isalnum() else " " for c in x ]))
model_df['verse'] = model_df['verse'].apply(lambda x: x.replace('  ',' '))

In [39]:
stopwords = ["yeah", "uh", "huh", "aw", "oh", "la", "ayy",
            "na", "ha", "ay", "skrrt",
            "em", "bla", "woo", "wu", "ya", "wala", "woop", "wrarrr", "ooh",
            "ba", "duh", "du", "1", "2", "3", "4" ,"5" ,"6", "7", "8", "9", "0"]

for index, row in model_df.iterrows():
    querywords = row['verse'].split()

    resultwords  = [word for word in querywords if word.lower() not in stopwords]
    row['verse'] = ' '.join(resultwords)

model_df.verse[:5]

1282    if you was in a pine box i would surely break ...
1284    they wishin and wishin and wishin and wishin t...
1285    i been movin calm dont start no trouble with m...
1286    gods plan gods plan i hold back sometimes i wo...
1287    and still bad things its a lot of bad things t...
Name: verse, dtype: object

In [40]:
verses = list(model_df.verse.values)

In [41]:
verses[:10]

['if you was in a pine box i would surely break the lock id jump right in and fall asleep cause you are the death of me if you was in a pine box i would surely break the lock id jump right in and fall asleep cause you are the death of me',
 'they wishin and wishin and wishin and wishin they wishin on me yuh',
 'i been movin calm dont start no trouble with me tryna keep it peaceful is a struggle for me dont pull up at am to cuddle with me you know how i like it when you lovin on me i dont wanna die for them to miss me yes i see the things that they wishin on me hope i got some brothers that outlive me they gon tell the story shit was different with me',
 'gods plan gods plan i hold back sometimes i wont yuh i feel good sometimes i dont dont i finessed down weston road nessed might go down a g o d wait i go hard on southside g yuh wait i make sure that north side eat',
 'and still bad things its a lot of bad things that they wishin and wishin and wishin and wishin they wishin on me bad t

In [42]:
all_verses_all_rappers = ','.join(verses)

all_verses_all_rappers[:100]

'if you was in a pine box i would surely break the lock id jump right in and fall asleep cause you ar'

In [43]:
file = all_verses_all_rappers

In [44]:
# Read in text and change unicode characters to ASCII
import unidecode
import string
import random
import re

all_characters = string.printable
n_characters = len(all_characters)

file_len = len(file)
print(f'file_len = {file_len}')

file_len = 718122


In [45]:
chunk_len = 400

def random_chunk():
    start_index = random.randint(0, file_len - chunk_len)
    end_index = start_index + chunk_len + 1
    return file[start_index:end_index]

print(random_chunk())

e at them bitches hating on you with their friends girl you got it let know that everything big nails done hair done everything big and my girls let me see your hands wave at them bitches hating on you with their friends girl you got it let know everything big nails done hair done everything big and my to girls let me see your hands wave at them bitches hating on you with their friends girl you got


In [46]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        
        self.encoder = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers)
        self.decoder = nn.Linear(hidden_size, output_size)
        
    def forward(self, input, hidden):
        output = self.encoder(input.view(1, -1))
        output, hidden = self.gru(output.view(1, 1, -1), hidden)
        output = self.decoder(output.view(1, -1))
        
        return output, hidden
    
    def init_hidden(self):
        return Variable(torch.randn(self.n_layers, 1, self.hidden_size))

In [47]:
def char2tensor(string):
    tensor = torch.zeros(len(string)).long()
    for char in range(len(string)):
        if string[char] in all_characters:
            tensor[char] = all_characters.index(string[char])
        else:
            tensor[char] = 94 #predict space if character unknown
        
    return Variable(tensor)

# Let's see it in action.
print(char2tensor('Metis0123abczABC'))

tensor([48, 14, 29, 18, 28,  0,  1,  2,  3, 10, 11, 12, 35, 36, 37, 38])


In [48]:
def random_training_set():
    chunk = random_chunk()
    inp = char2tensor(chunk[:-1])
    target = char2tensor(chunk[1:])
    return inp, target

In [49]:
def evaluate(model, prime_str='A', predict_len=100, temperature=0.8):
    
    hidden = model.init_hidden()
    prime_input = char2tensor(prime_str)
    predicted = prime_str
    
    # use priming string to build up hidden state
    
    for p in range(len(prime_str) - 1):
        _, hidden = model(prime_input[p], hidden)
    inp = prime_input[-1]
    
    for p in range(predict_len):
        output, hidden = model(inp, hidden)
        
        # Sample from the network as a multinomial distribution
        output_dist = output.data.view(-1).div(temperature).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        
        # Add predicted character to string and use as next input
        predicted_char = all_characters[top_i]
        predicted += predicted_char
        inp = char2tensor(predicted_char)
        
    return predicted

In [50]:
# helper function
import time, math

def time_since(since):
    s = time.time() - since
    m = math.floor(s/60)
    s -= m*60
    return '%dm %ds' % (m, s)

In [51]:
# The actual training part
def train(inp, target):
    hidden = model.init_hidden()
    model.zero_grad()
    loss = 0

    for char in range(chunk_len):
        output, hidden = model(inp[char], hidden)
        loss += criterion(output, target[char].unsqueeze(0))

    loss.backward()
    model_optimizer.step()

    return loss.data.item() / chunk_len

In [52]:
# parameters
# n_epochs = 1000
# print_every = 100
# plot_every = 10
# hidden_size = 256
# n_layers = 2
# learning_rate = 0.001

# # model declaration
# model = RNN(n_characters, hidden_size, n_characters, n_layers)
# model_optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
# criterion = nn.CrossEntropyLoss()

# start = time.time()
# all_losses = []
# loss_avg = 0

# for epoch in range(1, n_epochs + 1):
#     loss = train(*random_training_set())       
#     loss_avg += loss

#     if epoch % print_every == 0:
#         print('[%s (%d %d%%) %.4f]' % (time_since(start), epoch, epoch / n_epochs * 100, loss))
#         print(evaluate(model, 'A ', 100), '\n')

#     if epoch % plot_every == 0:
#         all_losses.append(loss_avg / plot_every)
#         loss_avg = 0

In [53]:
# import matplotlib.pyplot as plt
# import matplotlib.ticker as ticker
# %matplotlib inline

# plt.figure()
# plt.plot(all_losses)

In [54]:
# with open('models/drake_rnn_generator.pkl', 'wb') as picklefile:
#     pickle.dump(model, picklefile)

In [55]:
# Evaluate model from this notebook
print(evaluate(model, 'To be or not to be: ', predict_len=200, temperature=0.5))

To be or not to be: sound to like should on me schoole me one its up the back you got all goin and that your see of to not me run i dont im try started just both a for the bottom been started the bottom not me back both 


In [56]:
# Evaluate model from this notebook
print(evaluate(model, 'I miss you girl come back ', predict_len=200, temperature=0.5))

I miss you girl come back from the sure when your sweepin on everything on though the stried they feelens got on to shit i say the still on the better been im the motime and do the fines so the city nothing and got the real i 


In [57]:
# Evaluate model from this notebook
print(evaluate(model, "Started from the bottom", predict_len=200, temperature=0.4))

Started from the bottom the game when i dont so for the had that not on me for shout back and so the come that your better not the both im the stay for you want they for sturn im the star started on to me get the contert an


In [58]:
# Evaluate model from this notebook
print(evaluate(model, "I'm the realest rapper alive", predict_len=200, temperature=0.4))

I'm the realest rapper alive you stay i dont just started the could to the right the stressin it on the mornet the talkin of like a still of the grand they still be its my got the were you started the feeling im the really down 


In [59]:
# Evaluate model from this notebook
print(evaluate(model, "Smoke a lot of weed", predict_len=200, temperature=0.4))

Smoke a lot of weed thats its for me do the stried remix all be the fuck the fuck its for you want to the really mistake the fuckin playing i dont not got and i gotta started the star a man and the still and on the stil


In [60]:
# Evaluate model from this notebook
print(evaluate(model, "Baby come", predict_len=200, temperature=0.4))

Baby come i dont started its on the fuckin be should i like my fuckin on your part like the same and they stalf you still of the strist not still on my look and its all got and it im got the remix back on you 


In [61]:
prompts = ['I miss you girl come back ', 'To be or not to be: ', "Started from the bottom", "I'm the realest rapper alive",
           "Smoke a lot of weed", "Baby come", 'Date a scientist', 'Butter my biscuit', 'Scramble my ', 'I sleep ',
           'Green eggs and ', 'Rollin in my', 'Drop top', 'Math is fun', 'Sentences', 'Coffee in the', 'Table that',
           'Homies', 'From the streets', 'Maserati', 'Air force one', 'President', 'Trump']

drake_rnn_verses = []

for i in range(len(prompts)):
    drake_rnn_verses.append(evaluate(model, prompts[i], predict_len=200, temperature=0.5))
    
    
drake_rnn_verses

['I miss you girl come back me the slide in the gotta stery dont sention the both a lot thats the say you want the fucking got they see that yours i made am of the deates at thats shit i got for you know what yours this in my im',
 'To be or not to be: i take the romis it it how your off it see i dont night the strying the fay we got the days the fine you really for you got the striend not girl on the should i cant on the but the light all be the go',
 'Started from the bottom now its you got mind it they fither got starty the faction be been it so here so i like the how ive seever the got not not dont sound of my spit the like your be on list im side from now so stop i st',
 "I'm the realest rapper alivers and im to sure and should one to stay for a look all the got remember back its been yours like its of im the remix but i see the girl that i dont sound the put they not always i so what if the stil",
 'Smoke a lot of weed the for me how i real my for im not of my really send the sti

In [62]:
columns = ['generated_verses']

drake_generated_df = pd.DataFrame(drake_rnn_verses, columns=columns)

In [66]:
drake_generated_df['artist'] = 'Drake'

drake_generated_df

,generated_verses,artist
0,I miss you girl come back me the slide in the ...,Drake
1,To be or not to be: i take the romis it it how...,Drake
2,Started from the bottom now its you got mind i...,Drake
3,I'm the realest rapper alivers and im to sure ...,Drake
4,Smoke a lot of weed the for me how i real my f...,Drake
5,Baby come on of the really say you cant really...,Drake
6,Date a scientistic im dont stop i would do its...,Drake
7,Butter my biscuition could to man yours this s...,Drake
8,Scramble my every the got of and do the one no...,Drake
9,I sleep im not me on the bottom just sto stop ...,Drake


In [67]:
with open('data/drake_generated_raps.pkl', 'wb') as picklefile:
    pickle.dump(drake_generated_df, picklefile)